# Projeto da Disciplina BIG DATA PARA DADOS PÚBLICOS - 2022/2

### Aluno: Arthur Cisotto Machado



### 1. Introdução

O objetivo deste projeto consiste no uso de técnicas aprendidas em aula para conduzir um projeto de análise de grandes volumes de dados.  

O projeto foi desenvolvido utilizando o Dask, uma biblioteca Python que permite a manipulação de dados em paralelo, e o Pandas, uma biblioteca Python para manipulação de dados.  

O projeto consiste na análise de dados de avaliações de livros do site Goodreads (Amazon). O dataset utilizado pode ser encontrado no [Kaggle](https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews), e consiste em 2 arquivos CSV, sendo eles:  
- "books_data.csv" - contém informações sobre os livros, como título, autor, editora, etc.
- "Books_rating.csv" - contém informações sobre as avaliações dos livros, como nota, data da avaliação, texto da avaliação, etc.

### 2. Objetivos
A partir dos dados da base, realizar uma análise exploratória ressaltando padrões de comportamento nos livros e suas avaliações disponíveis na base (score, gênero, data de lançamento, autor).  
  
Também poderiam ser construídos modelos que busquem prever a avaliação média de um livro o com base no autor e gênero, por exemplo. Seria possível também criar um modelo de classificação da nota de um livro de 1 a 5 estrelas com base na descrição feita na avaliação.





### 3. Desenvolvimento

### Importando as Bibliotecas Necessárias para a execução do trabalho:



In [1]:
# Para utilizar recursos do sistema
import os
import sys

# Bibliotecas DASK
import dask
from dask.distributed import Client
import dask.dataframe as dd # similar ao pandas
import dask.multiprocessing

# Para plot
%matplotlib inline
import matplotlib.pyplot as plt

### Configurando um ambiente Dask

Agora iremos configurar um ambiente Dask para execução das nossas análises. Para isso, iremos criar um **client**

In [4]:
client = Client(n_workers=3, threads_per_worker=1, memory_limit='1GB')

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 3,Total memory: 2.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53379,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 3
Started: Just now,Total memory: 2.79 GiB
Comm: tcp://127.0.0.1:53405,Total threads: 1
Dashboard: http://127.0.0.1:53406/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:53384,


### Carregando os dados em DataFrames do Dask

In [10]:
dados_livros = dd.read_csv(f'{os.getcwd()}/Dataset/books_data.csv',
                       sep=',',
                       encoding='ISO-8859-1')

In [18]:
reviews = dd.read_csv(f'{os.getcwd()}/Dataset/Books_rating.csv',
                       sep=',',
                       encoding='ISO-8859-1',
                       dtype={'Id': 'category'})

In [22]:
dados_livros.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [19]:
reviews.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
